In [ ]:
toggle = False

def toggling():
    global toggle
    toggle = not toggle  # 상태 토글
    return gr.update(visible=toggle), gr.update(visible=not toggle)

In [ ]:
test11 = f"0번 : {completion.choices[0].message.context['intent'].split('", "')[0]}, 1번 : {completion.choices[0].message.context['intent'].split('", "')[1]}, 2번 : {completion.choices[0].message.context['intent'].split('", "')[2]}"    # 0번 : ["오토바이 수화, 1번 : 수화로 오토바이 표현, 2번 : 오토바이를 수화로 어떻게 표현하나요"] 영어로 나올 때도 있음

In [2]:
################# CHATBOT ################# 다은님 / 예외 메시지, 클리어
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()  # 속성 접근 방식으로 수정
        
        # 검색 결과 없는 기본 메시지 -> 원하는 메세지로 예외처리
        if "요청된 정보는 검색된 데이터에서 찾을 수 없습니다. 다른 쿼리나 주제를 시도해 보세요." in assistant_reply:
            assistant_reply = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"

        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        # 출처와 관련된 내용 확인
        try:
            citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
            video_url = citations[-2]
            image_urls = citations[-3].split(' ')  # 이미지 URL이 여러 개일 수 있음
        except (KeyError, IndexError):
            # 출처 정보를 찾을 수 없을 경우 예외 처리
            video_url = None
            image_urls = []

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        # speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, chat_history, video_url, image_urls

    except Exception as e:
        # 오류 메시지 대체
        fallback_message = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"
        chat_history.append((user_input, fallback_message))
        return chat_history, chat_history, None, []

In [ ]:
'''
원형님 코드 음성 중지
'''

import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 한글 분리 함수 (2번 코드)
def split_hangul(word):
    initial_consonants = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    medial_vowels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonants = final_consonants = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
 

    separated = []
    for char in word:
        if ord('가') <= ord(char) <= ord('힣'):  # 한글 음절인지 확인
            code = ord(char) - ord('가')
            initial = code // (21 * 28)  # 초성
            medial = (code % (21 * 28)) // 28  # 중성
            final = code % 28  # 종성

            separated.append(initial_consonants[initial])  # 초성 추가
            separated.append(medial_vowels[medial])  # 중성 추가
            if final_consonants[final]:  # 종성이 있다면 추가
                separated.append(final_consonants[final])
        else:
            separated.append(char)  # 한글이 아닌 문자는 그대로 추가

    return separated

# 한글 자음모음 & 커스텀비전 태그 연결 딕셔너리
hangul_dict = {"Giyeok": "ㄱ", "Nieun": "ㄴ", "Digeut": "ㄷ", "Rieul": "ㄹ", "Mieum": "ㅁ", "Bieup": "ㅂ", "Siot": "ㅅ",
        "Ieung": "ㅇ", "Jieut": "ㅈ", "Chieut": "ㅊ", "Kieuk": "ㅋ", "Tieut": "ㅌ", "Pieup": "ㅍ", "Hieut": "ㅎ",
        "A": "ㅏ", "Ya": "ㅑ", "Eo": "ㅓ", "Yeo": "ㅕ", "O": "ㅗ", "Yo": "ㅛ", "U": "ㅜ", "Yu": "ㅠ", "Eu": "ㅡ",
        "Yi": "ㅣ", "Ae": "ㅐ", "Yae": "ㅒ", "E": "ㅔ", "Ye": "ㅖ", "Oe": "ㅚ", "Wi": "ㅟ", "Ui": "ㅢ"}
reverse_hangul_dict = {value: key for key, value in hangul_dict.items()}  # 역딕셔너리 생성

# 음성 멈춤 기능을 위한 전역 변수
is_speaking = False

def stop_speech():
    """음성을 멈추는 함수"""
    global is_speaking
    if is_speaking:
        speech_synthesizer.stop_speaking_async()
        is_speaking = False
    return "Speech stopped."

# 채팅 함수 정의
def chat_with_openai(user_input, chat_history):
    try:
        global is_speaking

        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        assistant_reply = completion.choices[0].message.content.strip()
        chat_history.append((user_input, assistant_reply))

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        is_speaking = True
        speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, chat_history

    except Exception as e:
        return [("Error", str(e))], chat_history

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.Markdown("# Azure OpenAI + Cognitive Search + Speech 기반 수화 챗봇")
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
            stop_button = gr.Button("Stop Speech")  # 음성을 멈추는 버튼 추가
            decompose_button = gr.Button("Decompose Hangul")

        with gr.Column():
            hangul_output = gr.Textbox(label="Hangul Decomposition Result")

    state = gr.State([])  # 대화 기록 저장

    def decompose_hangul(user_input):
        """한글 자음/모음을 분리하는 함수"""
        separated_word = split_hangul(user_input)
        result_tags = [reverse_hangul_dict[char] for char in separated_word if char in reverse_hangul_dict]
        return ", ".join(result_tags)

    user_input.submit(chat_with_openai, [user_input, state], [chatbot, state])
    clear_button.click(lambda: [], None, chatbot)
    stop_button.click(stop_speech)  # 음성을 멈추는 버튼과 함수 연결
    decompose_button.click(decompose_hangul, inputs=user_input, outputs=hangul_output)

demo.launch()

In [4]:
pip install azure-cognitiveservices-vision-customvision

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install requests gradio matplotlib pillow opencv-python numpy azure-cognitiveservices-vision-customvision msrest

  Obtaining dependency information for python-multipart==0.0.12 from https://files.pythonhosted.org/packages/f5/0b/c316262244abea7481f95f1e91d7575f3dfcf6455d56d1ffe9839c582eb1/python_multipart-0.0.12-py3-none-any.whl.metadata
  Using cached python_multipart-0.0.12-py3-none-any.whl.metadata (1.9 kB)
Using cached python_multipart-0.0.12-py3-none-any.whl (23 kB)
  Attempting uninstall: python-multipart
    Found existing installation: python-multipart 0.0.20
    Uninstalling python-multipart-0.0.20:
      Successfully uninstalled python-multipart-0.0.20
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
'''
영준님 코드
이미지 폴더 필요
'''

import requests
import gradio as gr
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import cv2
import numpy as np
import io
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import time

ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"

# Initialize the prediction client
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

# 역딕셔너리 생성 (값 -> 키 변환용)
hangul_dict = {"Giyeok": "ㄱ", "Nieun": "ㄴ", "Digeut": "ㄷ", "Rieul": "ㄹ", "Mieum": "ㅁ", "Bieup": "ㅂ", "Siot": "ㅅ",
        "Ieung": "ㅇ", "Jieut": "ㅈ", "Chieut": "ㅊ", "Kieuk": "ㅋ", "Tieut": "ㅌ", "Pieup": "ㅍ", "Hieut": "ㅎ",
        "A": "ㅏ", "Ya": "ㅑ", "Eo": "ㅓ", "Yeo": "ㅕ", "O": "ㅗ", "Yo": "ㅛ", "U": "ㅜ", "Yu": "ㅠ", "Eu": "ㅡ",
        "Yi": "ㅣ", "Ae": "ㅐ", "Yae": "ㅒ", "E": "ㅔ", "Ye": "ㅖ", "Oe": "ㅚ", "Wi": "ㅟ", "Ui": "ㅢ"}

reverse_hangul_dict = {value: key for key, value in hangul_dict.items()}

# 단어를 문자로 나누는 함수
def split_hangul(word):
    # 한글 초성, 중성, 종성 리스트
    initial_consonants = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    medial_vowels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonants = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    separated = []  # 나눠진 문자가 들어갈 변수

    for char in word:
        if '가' <= char <= '힣':  # 한글 음절인지 확인
            code = ord(char) - ord('가')
            initial = code // (21 * 28)  # 초성
            medial = (code % (21 * 28)) // 28  # 중성
            final = code % 28  # 종성
            separated.append(initial_consonants[initial])  # 초성 추가
            separated.append(medial_vowels[medial])       # 중성 추가
            if final_consonants[final]:                   # 종성이 있다면 추가
                separated.append(final_consonants[final])
        else:
            separated.append(char)  # 한글이 아닌 문자는 그대로 추가

    return separated

# 지문자 확인 버튼을 누르면 나눠진 문자를 이미지로 보여주는 함수
def click_sign_send(sign_word):
    split_sign = split_hangul(sign_word)

    gallery_images = []
    sign_confirmed = []

    for sign in split_sign:
        sign_file_name = reverse_hangul_dict[sign]  # 영어 문자를 받는 변수 ex) Siot
        gallery_images.append(f"./images/Basic/{sign_file_name}.jpg")
        sign_confirmed.append(False)
    
    return gr.update(visible=False), gr.update(visible=True), gallery_images, sign_confirmed
        # output = [column1, column2, gallery, sign_confirmed_textbox]

#################################
# 카메라 인식 부분
#################################
def draw_boxes(image, predictions):
    """가장 확률이 높은 객체만 경계 상자를 그리도록 수정"""
    img = image.copy()
    
    # 예측 결과 중 확률이 가장 높은 하나를 선택
    if predictions:
        highest_prediction = max(predictions, key=lambda p: p.probability)
        
        # if first_char_is_inprogress:
            # if highest_prediction.tag_name == first_char and highest_prediction.probability > 0.7:
                # global first_char_succeed
                # first_char_succeed = True
                # print("First character detected successfully!")
        
        # 확률이 0.5 이상인 객체만 선택
        if highest_prediction.probability > 0.5:
            color = (255, 0, 0)  # 경계 상자 색상 (빨간색)
            box = highest_prediction.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])

            # 경계 상자 그리기
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
            
            # 라벨과 확률 텍스트 추가
            label = f"{highest_prediction.tag_name}: {highest_prediction.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img, highest_prediction.tag_name

def process_frame(frame, sign_word, sign_confirmed):

    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame, tag_name = draw_boxes(frame, results.predictions)

        gallery_images = []

        split_sign = split_hangul(sign_word)
        confirmed_list = list(sign_confirmed)

        i = 0
        for sign in split_sign:
            sign_file_name = reverse_hangul_dict[sign]
            sign_confirmed = confirmed_list[i]

            if sign_confirmed:
                gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
            else:
                if sign_file_name == tag_name:
                    gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                    confirmed_list[i] = True
                else :    
                    gallery_images.append(f"./images/Basic/{sign_file_name}.jpg") 
            i = i+1

        return annotated_frame, gallery_images, str(confirmed_list)
    
    except Exception as e:
        return frame, []

# 채팅
def request_gpt(user_text, temperature=0.7, top_p=0.95, max_tokens=800):

    # endpoint
    # method
    # header
    # payload

    api_base="https://team10-eighti.openai.azure.com" 
    deployment_id="gpt-4o"  
    endpoint = f"{api_base}/openai/deployments/{deployment_id}/chat/completions?api-version=2024-08-01-preview"
    api_key = '1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw'
    
    search_endpoint="https://team10-eighti-search.search.windows.net"
    search_key = 'wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3'
    search_index="sign-index"
    semantic_name = "sign-semantic"

    method = requests.post
    headers = {
        "Content-Type": "application.json",
        "api-key": api_key
    }
    payload = {
        "messages": [
            {
                "role": "system",
                "content": "당신은 사용자가 수화 정보를 찾는데 도움을 주는 수화 AI 도우미입니다."
            },
            {
                "role": "user",
                "content": user_text
            }
        ],
        "temperature": temperature,
        "top_p": top_p,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "max_tokens": max_tokens,
        "stop": None,
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": search_index,
                    "semantic_configuration": semantic_name,
                    "query_type": "semantic",
                    "fields_mapping": {},
                    "filter": None,
                    "top_n_documents": 5,
                    "authentication": {
                        "type": "api_key",
                        "key": search_key
                    }
                }
            }
        ]
    }

    response = requests.post(endpoint, headers=headers, json=payload)
    # print(response.status_code)
    # print(response.text)

    if response.status_code == 200:
        response_json = response.json()
        content_text = response_json['choices'][0]['message']['content']

        # citations = response_json['choices'][0]['message']['context']['citations'][0]['content'].split('\n')
        citations = response_json['choices'][0]['message']['context']['citations']
        if len(citations) > 0:
            citations_movie = get_citations(citations, "mp4")
            citations_images = get_citations(citations, "jpg").split(' ')
            citations_images_first = None
            citations_images_second = None

            if (len(citations_images) > 0 ):
                citations_images_first = get_image_url(citations_images[0])

                print(citations_images_first)
                if (len(citations_images) > 1 ):
                    citations_images_second = citations_images[1]
        else:
            citations_movie = "No Video"
            citations_images_first = None
            citations_images_second = None

        return content_text, citations_movie, citations_images_first, citations_images_second
    else:
        return "", "No Video", None, None

def get_citations(citations="", extentions="mp4"):
    if len(citations) > 0 and len(citations[0]) > 0 and len(citations[0]['content']) > 0:
        contents = citations[0]['content'].split('\n')

        for row in contents:
            row_list = row.split('.')
            if row_list[-1] == extentions:
                print("extentions=", extentions, "|row=",  row)
                return row

    return ""

def get_image_url(url):

    if not url.strip():
        return None  # 빈값 처리
    try:
        response = requests.get(url)
        response.raise_for_status()  # HTTP 에러 확인
        img = Image.open(BytesIO(response.content))  # 이미지를 메모리에서 읽음
        return img
    except Exception as e:
        return f"Error fetching image: {str(e)}"

# 사용자 입력을 전송하는 함수
def click_send(prompt, histories):
    # history_list = get_history_messages(histories=histories)
    # response_text, citation_html = request_gpt(prompt, history_list)
    response_text, citations_movie, citations_images_first, citations_images_second = request_gpt(prompt)
    histories.append((prompt, response_text))
    return histories, "", citations_movie, citations_images_first, citations_images_second, gr.update(visible=True), gr.update(visible=False)

with gr.Blocks() as demo:
    # 두 개의 상태
    col1_state = gr.State(value=True)  # Column 1 초기 visible 상태
    col2_state = gr.State(value=False)  # Column 2 초기 visible 상태

    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot(label='채팅 기록')
            with gr.Row():
                input_textbox = gr.Textbox(label="", scale=7)
                send_button = gr.Button("전송", scale=1)

        # citation = gr.HTML(label='참조')
        with gr.Column():
            with gr.Row():
                sign_input_textbox = gr.Textbox(label="지문자 입력 (2글자 단어)", value="소맥", scale=7)
                sign_confirmed_textbox = gr.Textbox(show_label=False, visible=True)
                toggle_button = gr.Button("지문자 확인", scale=1)

            with gr.Column(visible=True) as column1:
                videio = gr.Video(label="Video Player", autoplay=True)
                with gr.Row():
                    image_first = gr.Image(scale=0.3)
                    image_second = gr.Image(scale=0.3)

            with gr.Column(visible=False) as column2:
                with gr.Row():  # Row layout for input and output
                    gallery = gr.Gallery(columns=[6], rows=[1], show_label=False, show_share_button=False, show_download_button=False, interactive=False, show_fullscreen_button=False, height=130)
                with gr.Row():  # Row layout for input and output
                    with gr.Column():  # Input webcam column
                        webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
                    with gr.Column():  # Output display column
                        output = gr.Image(label="Detected Objects")

                # Set the process_frame function as the update function for webcam input
                webcam_input.stream(process_frame, inputs=[webcam_input, sign_input_textbox, sign_confirmed_textbox], outputs=[output, gallery, sign_confirmed_textbox])
                
                demo.title = "Azure Custom Vision Object Detection"
                demo.description = "Real-time object detection using Azure Custom Vision"

    input_textbox.submit(fn=click_send, inputs=[input_textbox, chatbot], outputs=[chatbot, input_textbox, videio, image_first, image_second, column1, column2])
    send_button.click(fn=click_send, inputs=[input_textbox, chatbot], outputs=[chatbot, input_textbox, videio, image_first, image_second, column1, column2])
    toggle_button.click(fn=click_sign_send, inputs=[sign_input_textbox], outputs=[column1, column2, gallery, sign_confirmed_textbox])

# content_text, citations_list = request_gpt("된장찌개가 수화로 어떻게 돼?", temperature=0.7, top_p=0.95, max_tokens=800)

# print(content_text)
# print(citations_list)

demo.launch(share=False)

c:\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\python\Lib\site-packages\gradio\components\chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(
c:\python\Lib\site-packages\gradio\components\base.py:201: UserWarning: 'scale' value should be an integer. Using 0.3 will cause issues.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\python\Lib\site-packages\gradio\queueing.py", line 669, in process_events
    time_limit=cast(int, fn.time_limit) - first_iteration
               ~~~~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~
TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'


In [14]:
'''
원형님이 만드신 최종본
'''

import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from gradio_modal import Modal
import cv2
from io import BytesIO
import requests
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import io
import time

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Initialize the prediction client
ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"

credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

# 역딕셔너리 생성 (값 -> 키 변환용)
hangul_dict = {"Giyeok": "ㄱ", "Nieun": "ㄴ", "Digeut": "ㄷ", "Rieul": "ㄹ", "Mieum": "ㅁ", "Bieup": "ㅂ", "Siot": "ㅅ",
        "Ieung": "ㅇ", "Jieut": "ㅈ", "Chieut": "ㅊ", "Kieuk": "ㅋ", "Tieut": "ㅌ", "Pieup": "ㅍ", "Hieut": "ㅎ",
        "A": "ㅏ", "Ya": "ㅑ", "Eo": "ㅓ", "Yeo": "ㅕ", "O": "ㅗ", "Yo": "ㅛ", "U": "ㅜ", "Yu": "ㅠ", "Eu": "ㅡ",
        "Yi": "ㅣ", "Ae": "ㅐ", "Yae": "ㅒ", "E": "ㅔ", "Ye": "ㅖ", "Oe": "ㅚ", "Wi": "ㅟ", "Ui": "ㅢ"}

reverse_hangul_dict = {value: key for key, value in hangul_dict.items()}

def split_hangul(word):
    # 한글 초성, 중성, 종성 리스트
    initial_consonants = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    medial_vowels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonants = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    separated = []
    for char in word:
        if '가' <= char <= '힣':  # 한글 음절인지 확인
            code = ord(char) - ord('가')
            initial = code // (21 * 28)  # 초성
            medial = (code % (21 * 28)) // 28  # 중성
            final = code % 28  # 종성
            separated.append(initial_consonants[initial])  # 초성 추가
            separated.append(medial_vowels[medial])       # 중성 추가
            if final_consonants[final]:                   # 종성이 있다면 추가
                separated.append(final_consonants[final])
        else:
            separated.append(char)  # 한글이 아닌 문자는 그대로 추가

    return separated

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 음성 멈춤 기능을 위한 전역 변수
is_speaking = False

# 음성을 멈추는 함수
def stop_speech():
    global is_speaking
    if is_speaking:
        speech_synthesizer.stop_speaking_async()
        is_speaking = False
    return "Speech stopped."

# 채팅
def chat_with_openai(user_input, chat_history):
    # 수어가 있는 경우
    try:
        global is_speaking
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()  # 속성 접근 방식으로 수정

        # 출처와 관련된 내용 확인
        citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
        video_url = citations[-2]
        image_urls = citations[-3].split(' ')  # 이미지 URL이 여러 개일 수 있음

        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        is_speaking = True
        speech_synthesizer.speak_text_async(assistant_reply)
        
        return chat_history, '', video_url, image_urls, gr.update(visible=True), gr.update(visible=False)

    # 지문자로 대체하는 경우
    except Exception as e:
        # 오류 메시지 대체
        fallback_message = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"
        chat_history.append((user_input, fallback_message))

        return chat_history, '', None, [], gr.update(visible=False), gr.update(visible=True)


def draw_boxes(image, predictions):
    """Draw bounding boxes on the image based on predictions"""
    img = image.copy()
    for pred in predictions:
        if pred.probability > 0.9 :
            color = (255, 0, 0)
            box = pred.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])
        
        # Draw rectangle
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
        
        # Add label with confidence score
            label = f"{pred.tag_name}: {pred.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img, pred.tag_name

def click_sign_send(sign_word):
    split_sign = split_hangul(sign_word)

    gallery_images = []
    sign_confirmed = []

    for sign in split_sign:
        sign_file_name = reverse_hangul_dict[sign]
        gallery_images.append(f"./images/Basic/{sign_file_name}.jpg")
        sign_confirmed.append(False)

    return gr.update(visible=False), gr.update(visible=True), gallery_images, sign_confirmed

# Custom Vision
def process_frame(frame, sign_word, sign_confirmed):
    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame, tag_name  = draw_boxes(frame, results.predictions)
        
        gallery_images = []

        split_sign = split_hangul(sign_word)
        confirmed_list = list(sign_confirmed)

        i = 0
        for sign in split_sign:
            sign_file_name = reverse_hangul_dict[sign]
            sign_confirmed = confirmed_list[i]

            if sign_confirmed:
                gallery_images.append(f"./images/Correct/{sign_file_name}.jpg")
            else:
                if sign_file_name == tag_name:
                    gallery_images.append(f"./images/Correct/{sign_file_name}.jpg")
                    confirmed_list[i] = True
                else : 
                    gallery_images.append(f"./images/Basic/{sign_file_name}.jpg")
                i = i+1
        
        return annotated_frame, gallery_images, str(confirmed_list)
    
    except Exception as e:
        return annotated_frame, []

# Gradio 인터페이스 구성
demo.title = "Azure Custom Vision Object Detection"
demo.description = "Real-time object detection using Azure Custom Vision"

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
            stop_button = gr.Button("Stop Speech")
        with gr.Column(visible=False) as column1:   # 기본적으로 숨김
            video_display = gr.Video(label="수어 영상")
            image_display = gr.Gallery(label="수어 이미지", columns=3)
        with gr.Column(visible=False) as column2:   # 기본적으로 숨김
            with gr.Row():
                sign_input_textbox = gr.Textbox(label="지문자 입력 (2글자 단어)", value="소맥", scale=7, max_length=2)
                sign_confirmed_textbox = gr.Textbox(show_label=False, visible=True)
                toggle_button = gr.Button("지문자 확인", scale=1)
            with gr.Row():  # Row layout for input and output
                gallery = gr.Gallery(columns=[6], rows=[1], show_label=False, show_share_button=False, show_download_button=False, interactive=False, show_fullscreen_button=False, height=130)
            with gr.Row():
                webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
                webcam_output = gr.Image(label="Detected Objects")
            show_btn = gr.Button("힌트")
            webcam_input.stream(process_frame, inputs=[webcam_input, sign_input_textbox, sign_confirmed_textbox], outputs=[webcam_output, gallery, sign_confirmed_textbox])
        
    with Modal(visible=False) as modal:  # 지문자 힌트 보여주기
            gr.Textbox("이미지가 보여질 공간")

    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    webcam_input.stream(process_frame, inputs=webcam_input, outputs=webcam_output)
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, user_input, video_display, image_display, column1, column2])
    clear_button.click(
        lambda: ([], []),  # chatbot과 state를 모두 초기화
        inputs=None,
        outputs=[chatbot, state]  # chatbot과 state 모두 업데이트
    )
    stop_button.click(stop_speech)  # 음성을 멈추기
    show_btn.click(lambda: Modal(visible=True), None, modal)  # 지문자 힌트 보여주기
    toggle_button.click(fn=click_sign_send, inputs=[sign_input_textbox], outputs=[column1, column2, gallery, sign_confirmed_textbox])

# 실행
demo.launch()

c:\python\Lib\site-packages\gradio\components\chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(
c:\python\Lib\site-packages\gradio\utils.py:980: UserWarning: Expected 3 arguments for function <function process_frame at 0x000001DBECFBE660>, received 1.
  warnings.warn(
c:\python\Lib\site-packages\gradio\utils.py:984: UserWarning: Expected at least 3 arguments for function <function process_frame at 0x000001DBECFBE660>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7895

To create a public link, set `share=True` in `launch()`.


In [18]:
'''
영준님 최종 코드
'''

import requests
import gradio as gr
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import cv2
import numpy as np
import io
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import time

ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"

# Initialize the prediction client
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

# 역딕셔너리 생성 (값 -> 키 변환용)
hangul_dict = {"Giyeok": "ㄱ", "Nieun": "ㄴ", "Digeut": "ㄷ", "Rieul": "ㄹ", "Mieum": "ㅁ", "Bieup": "ㅂ", "Siot": "ㅅ",
        "Ieung": "ㅇ", "Jieut": "ㅈ", "Chieut": "ㅊ", "Kieuk": "ㅋ", "Tieut": "ㅌ", "Pieup": "ㅍ", "Hieut": "ㅎ",
        "A": "ㅏ", "Ya": "ㅑ", "Eo": "ㅓ", "Yeo": "ㅕ", "O": "ㅗ", "Yo": "ㅛ", "U": "ㅜ", "Yu": "ㅠ", "Eu": "ㅡ",
        "Yi": "ㅣ", "Ae": "ㅐ", "Yae": "ㅒ", "E": "ㅔ", "Ye": "ㅖ", "Oe": "ㅚ", "Wi": "ㅟ", "Ui": "ㅢ"}

reverse_hangul_dict = {value: key for key, value in hangul_dict.items()}

def split_hangul(word):
    # 한글 초성, 중성, 종성 리스트
    initial_consonants = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    medial_vowels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonants = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    separated = []
    for char in word:
        if '가' <= char <= '힣':  # 한글 음절인지 확인
            code = ord(char) - ord('가')
            initial = code // (21 * 28)  # 초성
            medial = (code % (21 * 28)) // 28  # 중성
            final = code % 28  # 종성
            separated.append(initial_consonants[initial])  # 초성 추가
            separated.append(medial_vowels[medial])       # 중성 추가
            if final_consonants[final]:                   # 종성이 있다면 추가
                separated.append(final_consonants[final])
        else:
            separated.append(char)  # 한글이 아닌 문자는 그대로 추가

    return separated

def draw_boxes(image, predictions):
    """가장 확률이 높은 객체만 경계 상자를 그리도록 수정"""
    img = image.copy()
    
    # 예측 결과 중 확률이 가장 높은 하나를 선택
    if predictions:
        highest_prediction = max(predictions, key=lambda p: p.probability)
        
        # if first_char_is_inprogress:
        # logger.info(str(highest_prediction.tag_name) + " " + str(highest_prediction.probability))
            # if highest_prediction.tag_name == first_char and highest_prediction.probability > 0.7:
                # global first_char_succeed
                # first_char_succeed = True
                # print("First character detected successfully!")
        
        # 확률이 0.5 이상인 객체만 선택
        if highest_prediction.probability > 0.5:
            color = (255, 0, 0)  # 경계 상자 색상 (빨간색)
            box = highest_prediction.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])

            # 경계 상자 그리기
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
            
            # 라벨과 확률 텍스트 추가
            label = f"{highest_prediction.tag_name}: {highest_prediction.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img, highest_prediction.tag_name

def click_sign_send(sign_word):
    split_sign = split_hangul(sign_word)

    gallery_images = []
    sign_confirmed = []

    for sign in split_sign:
        sign_file_name = reverse_hangul_dict[sign]
        gallery_images.append(f"./images/Basic/{sign_file_name}.jpg")
        sign_confirmed.append(False)
    
    return gr.update(visible=False), gr.update(visible=True), gallery_images, sign_confirmed

#################################
# 카메라 인식 부분
#################################
def process_frame(frame, gallery_origin_images, sign_word, sign_confirmed):
    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame, tag_name = draw_boxes(frame, results.predictions)

        gallery_images = []

        split_sign = split_hangul(sign_word)
        sign_confirmed = sign_confirmed.replace("[","").replace("]", "")

        sign_confirmed_list = sign_confirmed.split(',')
        sign_confirmed_str_list = []

        for sign_confirmed in sign_confirmed_list:
            sign_confirmed = sign_confirmed.replace("'","").replace('"', "").strip()
            sign_confirmed_str_list.append(sign_confirmed)


        # for confirmed in sign_confirmed_str_list:
        #     if confirmed == "True":
        #         logger.info(str(sign_confirmed_str_list))

        confirmed_result= []
        for i in range(len(split_sign)):
            confirmed_result.append("False")


        i = 0
        for sign in split_sign:
            sign_file_name = reverse_hangul_dict[sign]
            confirmed = (sign_confirmed_str_list[i] == 'True')
            if confirmed:
                gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                confirmed_result[i] = "True"
            else:
                if sign_file_name == tag_name:
                    gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                    confirmed_result[i] = "True"
                else :    
                    gallery_images.append(f"./images/Basic/{sign_file_name}.jpg") 
                    confirmed_result[i] = "False"
            i = i+1

        return annotated_frame, gallery_images, str(confirmed_result)
    
    except Exception as e:
        return frame, gallery_origin_images, sign_confirmed


def request_gpt(user_text, temperature=0.7, top_p=0.95, max_tokens=800):

    # endpoint
    # method
    # header
    # payload

    api_base="https://team10-eighti.openai.azure.com" 
    deployment_id="gpt-4o"  
    endpoint = f"{api_base}/openai/deployments/{deployment_id}/chat/completions?api-version=2024-08-01-preview"
    api_key = '1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw'
    
    search_endpoint="https://team10-eighti-search.search.windows.net"
    search_key = 'wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3'
    search_index="sign-index"
    semantic_name = "sign-semantic"

    method = requests.post
    headers = {
        "Content-Type": "application.json",
        "api-key": api_key
    }
    payload = {
        "messages": [
            {
                "role": "system",
                "content": "당신은 사용자가 수화 정보를 찾는데 도움을 주는 수화 AI 도우미입니다."
            },
            {
                "role": "user",
                "content": user_text
            }
        ],
        "temperature": temperature,
        "top_p": top_p,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "max_tokens": max_tokens,
        "stop": None,
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": search_index,
                    "semantic_configuration": semantic_name,
                    "query_type": "semantic",
                    "fields_mapping": {},
                    "filter": None,
                    "top_n_documents": 5,
                    "authentication": {
                        "type": "api_key",
                        "key": search_key
                    }
                }
            }
        ]
    }

    response = requests.post(endpoint, headers=headers, json=payload)
    # print(response.status_code)
    # print(response.text)

    if response.status_code == 200:
        response_json = response.json()
        content_text = response_json['choices'][0]['message']['content']

        # citations = response_json['choices'][0]['message']['context']['citations'][0]['content'].split('\n')
        citations = response_json['choices'][0]['message']['context']['citations']
        if len(citations) > 0:
            citations_movie = get_citations(citations, "mp4")
            citations_images = get_citations(citations, "jpg").split(' ')
            citations_images_first = None
            citations_images_second = None

            if (len(citations_images) > 0 ):
                citations_images_first = get_image_url(citations_images[0])

                print(citations_images_first)
                if (len(citations_images) > 1 ):
                    citations_images_second = citations_images[1]
        else:
            
            citations_movie = "No Video"
            citations_images_first = None
            citations_images_second = None

        return content_text, citations_movie, citations_images_first, citations_images_second
    else:
        return "", "No Video", None, None

def get_citations(citations="", extentions="mp4"):
    if len(citations) > 0 and len(citations[0]) > 0 and len(citations[0]['content']) > 0:
        contents = citations[0]['content'].split('\n')

        for row in contents:
            row_list = row.split('.')
            if row_list[-1] == extentions:
                print("extentions=", extentions, "|row=",  row)
                return row

    return ""

def get_image_url(url):

    if not url.strip():
        return None  # 빈값 처리
    try:
        response = requests.get(url)
        response.raise_for_status()  # HTTP 에러 확인
        img = Image.open(BytesIO(response.content))  # 이미지를 메모리에서 읽음
        return img
    except Exception as e:
        return f"Error fetching image: {str(e)}"

def click_send(prompt, histories):
    # history_list = get_history_messages(histories=histories)
    # response_text, citation_html = request_gpt(prompt, history_list)
    response_text, citations_movie, citations_images_first, citations_images_second = request_gpt(prompt)
    histories.append((prompt, response_text))
    return histories, "", citations_movie, citations_images_first, citations_images_second, gr.update(visible=True), gr.update(visible=False)

with gr.Blocks() as demo:
    
    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot(label='채팅 기록')
            with gr.Row():
                input_textbox = gr.Textbox(label="", scale=7)
                send_button = gr.Button("전송", scale=1)

        # citation = gr.HTML(label='참조')
        with gr.Column():
            with gr.Row():
                sign_input_textbox = gr.Textbox(label="지문자 입력 (2글자 단어)", value="소맥", scale=7, max_length=2)
                sign_confirmed_textbox = gr.Textbox(show_label=False, visible=False)
                toggle_button = gr.Button("지문자 확인", scale=1)

            with gr.Column(visible=True) as column1:
                videio = gr.Video(label="Video Player", autoplay=True)
                with gr.Row():
                    image_first = gr.Image(scale=0.3)
                    image_second = gr.Image(scale=0.3)

            with gr.Column(visible=False) as column2:
                with gr.Row():  # Row layout for input and output
                    gallery = gr.Gallery(columns=[6], rows=[1], show_share_button=False, show_download_button=False, interactive=False, show_fullscreen_button=False, height=130, container=False)
                with gr.Row():  # Row layout for input and output
                    with gr.Column():  # Input webcam column
                        webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
                    with gr.Column():  # Output display column
                        output = gr.Image(label="Detected Objects")

                # Set the process_frame function as the update function for webcam input
                webcam_input.stream(process_frame, inputs=[webcam_input, gallery, sign_input_textbox, sign_confirmed_textbox], outputs=[output, gallery, sign_confirmed_textbox])
                
                demo.title = "Azure Custom Vision Object Detection"
                demo.description = "Real-time object detection using Azure Custom Vision"

    input_textbox.submit(fn=click_send, inputs=[input_textbox, chatbot], outputs=[chatbot, input_textbox, videio, image_first, image_second, column1, column2])
    send_button.click(fn=click_send, inputs=[input_textbox, chatbot], outputs=[chatbot, input_textbox, videio, image_first, image_second, column1, column2])
    toggle_button.click(fn=click_sign_send, inputs=[sign_input_textbox], outputs=[column1, column2, gallery, sign_confirmed_textbox])

# content_text, citations_list = request_gpt("된장찌개가 수화로 어떻게 돼?", temperature=0.7, top_p=0.95, max_tokens=800)

# print(content_text)
# print(citations_list)

demo.launch(share=False)

c:\python\Lib\site-packages\gradio\components\chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(
c:\python\Lib\site-packages\gradio\components\base.py:201: UserWarning: 'scale' value should be an integer. Using 0.3 will cause issues.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7899

To create a public link, set `share=True` in `launch()`.


extentions= mp4 |row= http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191028/632066/MOV000253800_700X466.mp4
extentions= jpg |row= http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151223/232818/IMG000227218_700X466.jpg http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151223/232819/IMG000227219_700X466.jpg
Error fetching image: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))


In [ ]:
'''
원형님이 만드신 최종본에 영준님 최종본 추가하기
'''

import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from gradio_modal import Modal
import cv2
from io import BytesIO
import requests
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import io
import time

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Initialize the prediction client
ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"

credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

# 역딕셔너리 생성 (값 -> 키 변환용)
hangul_dict = {"Giyeok": "ㄱ", "Nieun": "ㄴ", "Digeut": "ㄷ", "Rieul": "ㄹ", "Mieum": "ㅁ", "Bieup": "ㅂ", "Siot": "ㅅ",
        "Ieung": "ㅇ", "Jieut": "ㅈ", "Chieut": "ㅊ", "Kieuk": "ㅋ", "Tieut": "ㅌ", "Pieup": "ㅍ", "Hieut": "ㅎ",
        "A": "ㅏ", "Ya": "ㅑ", "Eo": "ㅓ", "Yeo": "ㅕ", "O": "ㅗ", "Yo": "ㅛ", "U": "ㅜ", "Yu": "ㅠ", "Eu": "ㅡ",
        "Yi": "ㅣ", "Ae": "ㅐ", "Yae": "ㅒ", "E": "ㅔ", "Ye": "ㅖ", "Oe": "ㅚ", "Wi": "ㅟ", "Ui": "ㅢ"}

reverse_hangul_dict = {value: key for key, value in hangul_dict.items()}

def split_hangul(word):
    # 한글 초성, 중성, 종성 리스트
    initial_consonants = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    medial_vowels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonants = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    separated = []
    for char in word:
        if '가' <= char <= '힣':  # 한글 음절인지 확인
            code = ord(char) - ord('가')
            initial = code // (21 * 28)  # 초성
            medial = (code % (21 * 28)) // 28  # 중성
            final = code % 28  # 종성
            separated.append(initial_consonants[initial])  # 초성 추가
            separated.append(medial_vowels[medial])       # 중성 추가
            if final_consonants[final]:                   # 종성이 있다면 추가
                separated.append(final_consonants[final])
        else:
            separated.append(char)  # 한글이 아닌 문자는 그대로 추가

    return separated

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 음성 멈춤 기능을 위한 전역 변수
is_speaking = False

# 음성을 멈추는 함수
def stop_speech():
    global is_speaking
    if is_speaking:
        speech_synthesizer.stop_speaking_async()
        is_speaking = False
    return "Speech stopped."

# 채팅
def chat_with_openai(user_input, chat_history):
    # 수어가 있는 경우
    try:
        global is_speaking
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()  # 속성 접근 방식으로 수정

        # 출처와 관련된 내용 확인
        citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
        video_url = citations[-2]
        image_urls = citations[-3].split(' ')  # 이미지 URL이 여러 개일 수 있음

        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        is_speaking = True
        speech_synthesizer.speak_text_async(assistant_reply)
        
        return chat_history, '', video_url, image_urls, gr.update(visible=True), gr.update(visible=False)

    # 지문자로 대체하는 경우
    except Exception as e:
        # 오류 메시지 대체
        fallback_message = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"
        chat_history.append((user_input, fallback_message))

        return chat_history, '', None, [], gr.update(visible=False), gr.update(visible=True)


def draw_boxes(image, predictions):
    """Draw bounding boxes on the image based on predictions"""
    img = image.copy()
    for pred in predictions:
        if pred.probability > 0.9 :
            color = (255, 0, 0)
            box = pred.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])
        
        # Draw rectangle
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
        
        # Add label with confidence score
            label = f"{pred.tag_name}: {pred.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img, pred.tag_name

def click_sign_send(sign_word):
    split_sign = split_hangul(sign_word)

    gallery_images = []
    sign_confirmed = []

    for sign in split_sign:
        sign_file_name = reverse_hangul_dict[sign]
        gallery_images.append(f"./images/Basic/{sign_file_name}.jpg")
        sign_confirmed.append(False)

    return gr.update(visible=False), gr.update(visible=True), gallery_images, sign_confirmed

# Custom Vision
def process_frame(frame, sign_word, sign_confirmed):
    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame, tag_name  = draw_boxes(frame, results.predictions)
        
        gallery_images = []

        split_sign = split_hangul(sign_word)
        confirmed_list = list(sign_confirmed)
        
        i = 0
        for sign in split_sign:
            sign_file_name = reverse_hangul_dict[sign]
            sign_confirmed = confirmed_list[i]

            if sign_confirmed:
                gallery_images.append(f"./images/Correct/{sign_file_name}.jpg")
            else:
                if sign_file_name == tag_name:
                    gallery_images.append(f"./images/Correct/{sign_file_name}.jpg")
                    confirmed_list[i] = True
                else : 
                    gallery_images.append(f"./images/Basic/{sign_file_name}.jpg")
                i = i+1
        
        return annotated_frame, gallery_images, str(confirmed_list)
    
    except Exception as e:
        return annotated_frame, []

# Gradio 인터페이스 구성
demo.title = "Azure Custom Vision Object Detection"
demo.description = "Real-time object detection using Azure Custom Vision"

with gr.Blocks() as demo:
    with gr.Row():

        with gr.Column():   # 채팅 섹션
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
            stop_button = gr.Button("Stop Speech")

        with gr.Column(visible=False) as column1:   # 수어 있는 경우 섹션, 기본적으로 숨김
            video_display = gr.Video(label="수어 영상")
            image_display = gr.Gallery(label="수어 이미지", columns=3)

        with gr.Column(visible=False) as column2:   # 지문자로 할 경우 섹션, 기본적으로 숨김

            with gr.Row():
                sign_input_textbox = gr.Textbox(label="지문자 입력 (2글자 단어)", value="소맥", scale=7, max_length=2)
                sign_confirmed_textbox = gr.Textbox(show_label=False, visible=True)
                toggle_button = gr.Button("지문자 확인", scale=1)

            with gr.Row():  # Row layout for input and output
                gallery = gr.Gallery(columns=[6], rows=[1], show_label=False, show_share_button=False, show_download_button=False, interactive=False, show_fullscreen_button=False, height=130)

            with gr.Row():
                webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
                webcam_output = gr.Image(label="Detected Objects")
            show_btn = gr.Button("힌트")
            webcam_input.stream(process_frame, inputs=[webcam_input, sign_input_textbox, sign_confirmed_textbox], outputs=[webcam_output, gallery, sign_confirmed_textbox])
        
    with Modal(visible=False) as modal:  # 지문자 힌트 보여주기
            gr.Textbox("이미지가 보여질 공간")

    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    webcam_input.stream(process_frame, inputs=webcam_input, outputs=webcam_output)
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, user_input, video_display, image_display, column1, column2])
    clear_button.click(
        lambda: ([], []),  # chatbot과 state를 모두 초기화
        inputs=None,
        outputs=[chatbot, state]  # chatbot과 state 모두 업데이트
    )
    stop_button.click(stop_speech)  # 음성을 멈추기
    show_btn.click(lambda: Modal(visible=True), None, modal)  # 지문자 힌트 보여주기
    toggle_button.click(fn=click_sign_send, inputs=[sign_input_textbox], outputs=[column1, column2, gallery, sign_confirmed_textbox])

# 실행
demo.launch()

In [ ]:
'''
원형님 마지막 코드 공유
'''

import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from gradio_modal import Modal
import cv2
from io import BytesIO
import requests
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from loguru import logger
import numpy as np
import io
import time

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Initialize the prediction client
ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"
 
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

# 역딕셔너리 생성 (값 -> 키 변환용)
hangul_dict = {"Giyeok": "ㄱ", "Nieun": "ㄴ", "Digeut": "ㄷ", "Rieul": "ㄹ", "Mieum": "ㅁ", "Bieup": "ㅂ", "Siot": "ㅅ",
        "Ieung": "ㅇ", "Jieut": "ㅈ", "Chieut": "ㅊ", "Kieuk": "ㅋ", "Tieut": "ㅌ", "Pieup": "ㅍ", "Hieut": "ㅎ",
        "A": "ㅏ", "Ya": "ㅑ", "Eo": "ㅓ", "Yeo": "ㅕ", "O": "ㅗ", "Yo": "ㅛ", "U": "ㅜ", "Yu": "ㅠ", "Eu": "ㅡ",
        "Yi": "ㅣ", "Ae": "ㅐ", "Yae": "ㅒ", "E": "ㅔ", "Ye": "ㅖ", "Oe": "ㅚ", "Wi": "ㅟ", "Ui": "ㅢ"}

reverse_hangul_dict = {value: key for key, value in hangul_dict.items()}

def split_hangul(word):
    # 한글 초성, 중성, 종성 리스트
    initial_consonants = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    medial_vowels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonants = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
 
    separated = []
    for char in word:
        if '가' <= char <= '힣':  # 한글 음절인지 확인
            code = ord(char) - ord('가')
            initial = code // (21 * 28)  # 초성
            medial = (code % (21 * 28)) // 28  # 중성
            final = code % 28  # 종성
            separated.append(initial_consonants[initial])  # 초성 추가
            separated.append(medial_vowels[medial])       # 중성 추가
            if final_consonants[final]:                   # 종성이 있다면 추가
                separated.append(final_consonants[final])
        else:
            separated.append(char)  # 한글이 아닌 문자는 그대로 추가
 
    return separated

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 음성 멈춤 기능을 위한 전역 변수
is_speaking = False

# 음성을 멈추는 함수
def stop_speech():
    global is_speaking
    if is_speaking:
        speech_synthesizer.stop_speaking_async()
        is_speaking = False
    return "Speech stopped."

# 채팅
def chat_with_openai(user_input, chat_history):
    # 수어가 있는 경우
    try:
        global is_speaking
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()  # 속성 접근 방식으로 수정

        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        # 출처와 관련된 내용 확인
        citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
        video_url = citations[-2]
        image_urls = citations[-3].split(' ')  # 이미지 URL이 여러 개일 수 있음

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        is_speaking = True
        speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, '', video_url, image_urls, gr.update(visible=True), gr.update(visible=False)

    # 지문자로 대체하는 경우
    except Exception as e:
        # 오류 메시지 대체
        fallback_message = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"
        chat_history.append((user_input, fallback_message))

        return chat_history, '', None, [], gr.update(visible=False), gr.update(visible=True)


def draw_boxes(image, predictions):
    """Draw bounding boxes on the image based on predictions"""
    img = image.copy()
    for pred in predictions:
        if pred.probability > 0.9 :
            color = (255, 0, 0)
            box = pred.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])
        
        # Draw rectangle
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
        
        # Add label with confidence score
            label = f"{pred.tag_name}: {pred.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img, pred.tag_name

def click_sign_send(sign_word):
    split_sign = split_hangul(sign_word)
    logger.info(sign_word + " " + str(split_sign))

    gallery_images = []
    sign_confirmed = []

    for sign in split_sign:
        sign_file_name = reverse_hangul_dict[sign]
        gallery_images.append(f"./images/Basic/{sign_file_name}.jpg")
        sign_confirmed.append(False)
    
    return gr.update(visible=False), gr.update(visible=True), gallery_images, sign_confirmed

# Custom Vision
def process_frame(frame, gallery_origin_images, sign_word, sign_confirmed):
    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame, tag_name = draw_boxes(frame, results.predictions)

        gallery_images = []

        split_sign = split_hangul(sign_word)
        # logger.info(sign_confirmed)
        sign_confirmed = sign_confirmed.replace("[","").replace("]", "")
        # logger.info(sign_confirmed)

        sign_confirmed_list = sign_confirmed.split(',')
        sign_confirmed_str_list = []

        for sign_confirmed in sign_confirmed_list:
            sign_confirmed = sign_confirmed.replace("'","").replace('"', "").strip()
            sign_confirmed_str_list.append(sign_confirmed)
            # logger.info(sign_confirmed)

        logger.info(sign_confirmed_str_list)

        # for confirmed in sign_confirmed_str_list:
        #     if confirmed == "True":
        #         logger.info(str(sign_confirmed_str_list))

        confirmed_result= []
        for i in range(len(split_sign)):
            confirmed_result.append("False")


        i = 0
        for sign in split_sign:
            sign_file_name = reverse_hangul_dict[sign]
            confirmed = (sign_confirmed_str_list[i] == 'True')
            # logger.info(confirmed)            
            if confirmed:
                gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                confirmed_result[i] = "True"
            else:
                if sign_file_name == tag_name:
                    gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                    confirmed_result[i] = "True"
                else :    
                    gallery_images.append(f"./images/Basic/{sign_file_name}.jpg") 
                    confirmed_result[i] = "False"
            i = i+1

        return annotated_frame, gallery_images, str(confirmed_result)
    
    except Exception as e:
        logger.error(f"Error during prediction: {e}")
        return frame, gallery_origin_images, sign_confirmed


# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
            stop_button = gr.Button("Stop Speech")
        with gr.Column(visible=False) as column1:   # 기본적으로 숨김
            video_display = gr.Video(label="수어 영상")
            image_display = gr.Gallery(label="수어 이미지", columns=3)
        with gr.Column(visible=False) as column2:   # 기본적으로 숨김
            with gr.Row():
                sign_input_textbox = gr.Textbox(label="지문자 입력 (2글자 단어)", value="", scale=7)
                sign_confirmed_textbox = gr.Textbox(show_label=False, visible=True)
                toggle_button = gr.Button("지문자 확인", scale=1)
            with gr.Row():  # Row layout for input and output
                gallery = gr.Gallery(columns=[6], rows=[1], show_label=False, show_share_button=False, show_download_button=False, interactive=False, show_fullscreen_button=False, height=130)
            with gr.Row():
                webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
                webcam_output = gr.Image(label="Detected Objects")
            show_btn = gr.Button("힌트")
            webcam_input.stream(process_frame, inputs=[webcam_input, gallery, sign_input_textbox, sign_confirmed_textbox], outputs=[webcam_output, gallery, sign_confirmed_textbox])
            demo.title = "Azure Custom Vision Object Detection"
            demo.description = "Real-time object detection using Azure Custom Vision"
        
    with Modal(visible=False) as modal:  # 지문자 힌트 보여주기
            gr.Textbox("이미지가 보여질 공간")


    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, user_input, video_display, image_display, column1, column2])
    clear_button.click(
        lambda: ([], []),  # chatbot과 state를 모두 초기화
        inputs=None,
        outputs=[chatbot, state]  # chatbot과 state 모두 업데이트
    )
    stop_button.click(stop_speech)  # 음성을 멈추기
    show_btn.click(lambda: Modal(visible=True), None, modal)  # 지문자 힌트 보여주기
    toggle_button.click(fn=click_sign_send, inputs=[sign_input_textbox], outputs=[column1, column2, gallery, sign_confirmed_textbox])

# 실행
demo.launch(share=False)